In [1]:
import pandas as pd

In [2]:
from functools import reduce

In [3]:
df_rnastralign = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/e2efold/data/rnastralign.pkl.gz', compression='gzip')

In [4]:
df_rfam = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/rna_cg/data/rfam.pkl')

In [5]:
df_s_processed_1 = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/rna_cg/data/s_processed.pkl')
df_s_processed_2 = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/rna_cg/data/s_processed_test.pkl')
df_s_processed = pd.concat([df_s_processed_1, df_s_processed_2])



In [6]:
df_pdb_250 = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/pdb_250/data/pdb_250.pkl')

In [7]:
df_bprna = pd.read_pickle('~/work/psi-lab-sandbox/rna_ss/data_processing/spot_rna/bp_rna/data/bp_rna.pkl.gz')

In [8]:
df_rfam = df_rfam.rename(columns={'seq_id': 'seq_id_rfam'})

In [9]:
df_rnastralign = df_rnastralign.rename(columns={'seq_id': 'seq_id_rnastralign'})

In [10]:
df_s_processed = df_s_processed.rename(columns={'seq_id': 'seq_id_sprocessed'})

In [11]:
df_pdb_250 = df_pdb_250.rename(columns={'seq_id': 'seq_id_pdb250'})
 

In [12]:
df_bprna = df_bprna.rename(columns={'seq_id': 'seq_id_bprna'})

In [14]:
len(df_rfam)

151

In [15]:
len(df_rnastralign)

37138

In [47]:
len(df_s_processed)

6247

In [17]:
len(df_pdb_250)

241

In [18]:
len(df_bprna)

13419

In [50]:
# de-dup!

df_rfam = df_rfam.drop_duplicates(subset=['seq'])
df_rnastralign = df_rnastralign.drop_duplicates(subset=['seq'])
df_s_processed = df_s_processed.drop_duplicates(subset=['seq'])
df_pdb_250 = df_pdb_250.drop_duplicates(subset=['seq'])
df_bprna = df_bprna.drop_duplicates(subset=['seq'])


In [51]:
len(df_rfam)

151

In [52]:
len(df_rnastralign)

30421

In [53]:
len(df_s_processed)    # this dataset has duplicates!!!!

5253

In [54]:
len(df_pdb_250)

241

In [55]:
len(df_bprna)

13419

In [56]:
# join on sequence
df = reduce(lambda left,right: pd.merge(left,right,on='seq',how='outer'), 
           [df_rfam, df_rnastralign, df_s_processed, df_pdb_250, df_bprna])


In [57]:
names = ['rfam', 'rnastralign', 'sprocessed', 'pdb250', 'bprna']

for i in range(len(names)):
    for j in range(i+1, len(names)):
        print(names[i], names[j], len(df[~df['seq_id_{}'.format(names[i])].isna() & ~df['seq_id_{}'.format(names[j])].isna()]))

rfam rnastralign 4
rfam sprocessed 1
rfam pdb250 0
rfam bprna 30
rnastralign sprocessed 287
rnastralign pdb250 11
rnastralign bprna 655
sprocessed pdb250 16
sprocessed bprna 318
pdb250 bprna 1


In [58]:
# debug
len(df[~df['seq_id_sprocessed'].isna()])   

5253

In [59]:
# debug
len(df[~df['seq_id_pdb250'].isna()])   

241

In [28]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
# non-exact sequence match, alignment-based
# to save computing time, only align sequence with comparable size

In [62]:
# seqs1 = df_pdb_250.seq.tolist()
seqs1 = df_rfam.seq.tolist()
# seqs2 = df_s_processed.seq.tolist()
seqs2 = df_pdb_250.seq.tolist()
n_similar = 0
n_identical = 0

for s1 in seqs1:
    # finding best hit for s1
    # TODO should also start with s1
    print(s1)
    print(len(s1))
    best_score = 0
    best_seq = None
    best_alignment = None
    for s2 in seqs2:
        # for debug
        if s2 == s1:
            n_identical += 1
        
        # only align sequence with comparable length
        if not(0.7 * len(s1) <=len(s2) <= 1.3* len(s1)):
            continue
        # these penalty term ensure that best case score = seq_len (when 2 seqs are identical)
        alignments = pairwise2.align.globalms(s1, s2, 1, -0.5, -1, -.1,  # match, mismatch, gap open, gap extension
                                              one_alignment_only=True)
        score = alignments[0][2]
        if score > best_score:
            best_score = score
            best_seq = s2
            best_alignment = alignments[0]
    if best_score > 0.8 * len(s1):
        print(format_alignment(*best_alignment))
        print(best_score)
        n_similar += 1
    else:
        print(format_alignment(*best_alignment))
        print(best_score)
        print("No similar sequence found!\n")

print("Found {} similar out of a total {}".format(n_similar, len(seqs1)))
print("[debug] n_identical {}".format(n_identical))
        

UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACCCGGUAGUUAAGCCCGCCAGCGCCGAUGGUACUGCACUGGUGACGGUGUGGGAGAGUAGGUCGCCGCCGGAC
117
--U--UCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACCCGGUAGUUAAGCCCGCCAGCGCCGAUGGUACUGCACUGGUGACGGUGUGGGAGAGUAGGUCG--CCGCCGGAC
  |  .|.|||...|||||||.|.||||.||||||.|||||||||||||.|||.|||.||.|.||||||||||||||||||||..|.||.|||.|..|||||||||||||||  .||..||| 
AAUCCCCCGUGCCCAUAGCGGCGUGGAACCACCCGUUCCCAUUCCGAACACGGAAGUGAAACGCGCCAGCGCCGAUGGUACUGGGCGGGCGACCGCCUGGGAGAGUAGGUCGGUGCGGGGGA-
  Score=75.7

75.7
No similar sequence found!

UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGCAGCUUGCUGCGUUACUUACCACGAAUUGCAGACGCUUAGAGUGGUGAAAUUUCGAACGCAUAGCACCAACUGGGCCUCCAGUUGGUACGUCUGGUUCAGGGUUG
150
UA---GCU-UC---AG-CGAU----GGAUCGGUUGCAUCG--AGUAUCGAUG---AAGAACGCAGCUUGCUGC------GUU----ACUUACCACGAAUUGCAGACGCUUAGA---GUGGUGAA--AUUUCG-AA----CG-CAUA---GCACCA-ACUG---GGCC-U--C-CAGUUG-GU-------AC-GUCUGGUUC---AGGGUUG
.|   |.| ||   || |.||    ||.|.|||||    |  |||||| |||   ||..|| ||   ||.|||      |||    ||||  

AGCUUUGCGCAGUGGCAGUAUCGU-AGCCAAUGAGG-UC---UAUCCGAG--GCGCGAUUAUUGCU----AAUUGAAAACUUUUCCCAAUACCCCGCC-----GU-----GA------CGACUUGCAA-------UAUAGUCGGCA-CUGGCAAU--UU----U-UG----ACAGU-------CUCUACG---GA-G---A
           | |||  | |||| ||   .||||| ||   ||| .|||  .|||      ||||    ||   |||   |.|||.||.|   ||||     ||     ||      ||||.|  ||       |.||.|  ||| ||||  ||  ||    | ||    |||||       |||||||   || |   |
-----------G-GGC--U-UCGUUAG---GUGAGGCUCCUGUAU-GGAGAUACGC------UGCUGCCCAA---AAA---UGUCCAAAGA---CGCCAAUGGGUCAACAGAAAUCAUCGACAU--AAGGUGAUUUUUAAU--GCAGCUGG--AUGCUUGUCCUAUGCCAUACAGUGCUAAAGCUCUACGAUUGAAGCCCA
  Score=49.6

49.599999999999945
No similar sequence found!

GACGGGCUUAGCAACGUGGGCCUGUAACCCAAGUGGGGGCAUGUGGGAAAUGGGACUUUGGGUCAACCUAGUGGAUCGGGUCCAGUGUUAGCUGCUUACUGGUCUGCCCAUUCCAAGCCGGGAGUUGGGCUGAGUGACCUGGGCGAAGGCUGGGUUGCGCAGCUCCUAGAGUGGAGGGCAAUGCGUGAGGCUGGCUUCACAGAGCAGCGACUACCUCCCGCUCUCGGCAGUGGAAGGAUAACGGGCCGGUGCUACCUGGAUCCACCAUGCUUCACUGGGCUGACUCUUAAUAGGACCAUUUCUUU
305
G--------ACGGG------CU-

UGCAGUUUGCUGCGCUAUUAGUUUGGAACA----AC-ACUGAGAAGAUUAGCAUGGCC---CCUGCGC-AAGGAC--G-G--CAU---CUUU---CU-----U-UG-AGAGGUGUGC---------UGGGCU--CGCCCAGCUUU-U------------
|.|||          ||       ||||||    || ||||    |||.|.||..|.|   .||||.| ||||.|  | |  |||   |.||   ||     | || ||||||| ||         ||||.|  ..||||.|.|| |            
UCCAG----------UA-------GGAACACUAUACUACUG----GAUAAUCAAAGACAAAUCUGCCCGAAGGGCUUGAGAACAUACCCAUUGCACUCCGGGUAUGCAGAGGUG-GCAGCCUCCGGUGGGUUAAAACCCAACGUUCUCAACAAUAGUGA
  Score=38.2

38.19999999999996
No similar sequence found!

AUGACUCUCUAAAUUGCAAAAUUUACCUUUGGAGGGAAAAGUUAUCAGGCCUGCACCUGAUAGCUAGUCUUUAAACCAAUAGAUUGCAUCGGUUUAAUAGGCAAGACCGUCAAAUUGCGGGAAAAGGGUCAACAGCCGUUCAGUACCAAGUCUCAGGGGAAACUUUGAGAUGGCCUUGCAAAGGAUAUGGUAAUAAGCUGACGGACAGGGUCCUAACCACGCAGCCAAGUCCUAAGUCAACAUUUCGGUGUUGAUAUGGAUGCAGUUCACAGACUAAAUGUCGGUCGGGGAAGAAUAGGUAUUCUUCUCAUAAGAUAUAGUCGGACCUCUCCUUAAUGGGAGCU
344
-----------AUG-----ACUCUCUAAAUUGCAAAAUU---UACCUUUGGAGGGAAAAGUUAUCAG---GCCU---GCACCUGAUAGC--UAGUCUUUAAACCAAUAGAU

GCGUUCUUGUCGGGGUGCC-----UUGCUA-U---GAGGCUGAGAUCGGUUAA-ACCGG-AUCCCGUUGAACCUGAUCAGGUUAGCGCCUGCGUAGGGAACAAGAUUUCUCGUCUC
||| .|   ||||||||||     |||| | |   ||||||||||      || |||.| |||       ||||||||.||.||..|||.||||||||||           |||.|
GCG-AC---UCGGGGUGCCCUCCAUUGC-ACUCCGGAGGCUGAGA------AAUACCCGUAUC-------ACCUGAUCUGGAUAAUGCCAGCGUAGGGAA-----------GUCGC
  Score=50.1

50.09999999999997
No similar sequence found!

CAGAAAGCGUCUAGCCAUGGCGCUAGUAUGAGUGUCGUGCAGCCUCCAGGACCCCCCCUCCCGGGAGAGCCAUAGUGGUCUGCGGAACCGGUGAGUACACCGGAAUUGCCAGGACGACCGGGUCCUUUCUUGGAUCAACCCGCUCAAUGCCUGGAGAUUUGGGCGUGCCCCCGCGAGACUGCUAGCCGAGUAGUGUUGGGUCGCGAAAGGCCUUGUGGUACUGCCUGAUAGGGUGCUUGCGAGUGCCCCGGGAGGUCUCGUAGACCGUGCACCAUGAGCACGAAUCCUAAACCUCAAAGAAAAACCAAACGUAACACCAACCG
323
CAGAAAGCGU-CUAGCCAUG---GC-G-CUAGUA---UGAGUGUC-------GU----GC-------------AGCCU--C-CAGGAC--CCCC-------CCU--CCCGGGA--GA-GC------CAUA--GU-GGUCUGCGGAAC-CG--------GUGA------GUA-CACCGGA--A--UUGC----CAGGACGACCGGGUCCUUUC--------UUGGAU-CA-ACCCGCUCAAUGCCUGGAGAUU--UG--GGC

GCACUCUCUA--UCAGAUUGGAU-GU-CUUGCUGCUAU-AAUAGA----UAGAGAAGGUU------------AUAGCAGACUA
||| |||.||  |||| .||||. || ||  |.||||. ||..||    |.| | |||||            || ||  ||.|
GCA-UCUGUAGCUCAG-CUGGAGAGUACU--CGGCUACGAACCGAGCGGUCG-G-AGGUUCGAACCUCCCGGAU-GC--ACCA
  Score=24.8

24.79999999999998
No similar sequence found!

AAUGAAUCACGCUGACUCACUCAUAUAAUCGCAAGAAUACGGCUUGCGAGUCUCUACCAGCCGACCGUAAAUCGGCUGACUAUGAGUGGCAAUGUCAGGA
100
-----AAU----------GAAUCACGCUG-ACUCACUCAU---AU-AAUCGCAAG-AA-----UACGGCUUGCGAGUCUCUACCAGCC-GA--CCGUAAAUCGGCUGAC----UAUGAGUGGCAAUGUCAGG--A
     |||          |||  |||..| ||.||   ||   .| ||||.||.| ||     ||.||||    |||||||..|.||| ||  |||    ||.|||.||    ||.|.|||| ||.|  |||  |
CGCUGAAUUCCCUGAAGGGAA--ACGGGGGACCCA---AUGGGGUGAAUCCCACGUAAGUGGGUAGGGCU----AGUCUCUUUCGGCCCGAACCCG----UCAGCUAACCUCGUAAGCGUGG-AAAG--AGGGAA
  Score=41.1

41.09999999999999
No similar sequence found!

CGCAUAAAUAGAGGAGCUGCCAAGCAUGUAUUUGGCGAGGUGUUAAGGAGAAGAACCUCCAAUACUCGCUGAAGAAGGUUUGGCUGCCGAAAGGGUGAGCUUGUUCUUGAGCUCA

UGGA-UCAUUGAUGAACAAAAAAAAAAACA---UCUGGGAGUC--CUCUGAGACAUCCAUGAUGACCACA------ACAUUGGG---AGUCUGAGGU---------------CCAC
.||. |.|..||||..||.||...|...||   |.||||| ||  .||||.|.|.||   |.||  ||||      ||| ||.|   |||| |||||               ||.|
GGGGCUGAAGGAUGCCCAGAAGGUACCCCAUUGUAUGGGA-UCUGAUCUGGGGCCUC---GGUG--CACAUGCUUUACA-UGUGUUUAGUC-GAGGUUAAAAAACGUCUAGGCCCC
  Score=31.3

31.29999999999997
No similar sequence found!

GAGGGUCAGGAUGUCAUCUUCAGCCAAUUCGACAGCCUGAGAGUCAGCUCUAACCAGAGCCCACAAGGUAAUCUGGGGAGUAAUGAACCCGGUAAUGUCGGUGGUAAUGAUGCUCUGGCAACCUCCACU
129
GAG--------GGUCAGG---AUGUCAU---------CUUCAGCC----AAU-UC----GACAGCC--UGAGAGUCAGCUCUAACCA--GAGCCCACAAGGUAAUCUGGGGAGUAAUGAACCCGG----UAAUGUC----G-------GUGGUAAUGAUGCUCU--G---GCAACCUCCACU
|.|        |||.|||   .||| ||         ||.|.|||    ||| ||    ||| |||  || | ||||.|...||.||  ||   ||.|||||.||.|     .|||||.|.|.||    |  ||||    |       |||.|||  | |||||  |   | ||.| |||  
GGGCUUCGUUAGGUGAGGCUCCUGU-AUGGAGAUACGCUGCUGCCCAAAAAUGUCCAAAGAC-GCCAAUG-G-GUCAACAGAAAUCAUC

UGUAACGGGGCAAGGCU-UGACC-GAGGGGACUAUAACAUGUA-------UAGGCGAAAAGCGGGGUCU-CGGUUG-UA-ACGC---GCU-UAGGA-AGUCCCCUCGAGG
.|.|||.|.|| ||||. ||||| |||.||  |    ||||.|       |  ||| |||||||   |. |.|.|| || ||||   .|| |.||| |||.|||   |  
GGGAACUGAGC-AGGCAAUGACCAGAGCGG--U----CAUGCAGCCGGGCU--GCG-AAAGCGG---CAACAGAUGAUACACGCACAUCUGUGGGACAGUUCCC---A--
  Score=36.7

36.69999999999998
No similar sequence found!

AGCGAGUAAGACAGACUCUUCAGUCUGAGUUUGUGGAGAUGAGUGUAAAUCUGGCAUAGCAUACAGGUUA
70
AGCGAGUAAGACAGACUC---UUCAGUCUGAGU---U----UGU-G---------GA---GAUGAG---UGUAAAUC-UGGCAUA-GCAUACAGGUUA
.|.|||| ||| .||.||   ||.|||.||.||   |    ||| |         ||   || |||   .||.|||| |.||||. || |.||||   
GGAGAGU-AGA-UGAUUCGCGUUAAGUGUGUGUGAAUGGGAUGUCGUCACACAACGAAGCGA-GAGCGCGGUGAAUCAUUGCAUCCGC-UCCAGG---
  Score=28.9

28.900000000000002
No similar sequence found!

AAUUCCAGCGAGAGGCAGAGGGAGCGAGCGGGCGGCCGGCUAGGGUGGAAGAGCCGGGCGAGCAGAGCUGCGCUGCGGGCGUCCUGGGAAGGGAGAUCCGGAGCGAAUAGGGGGCUUCGCCUCUGGCCCAGCCCUCCCGCUGAUCCCCCAGCCAGCGGUCCGCAACCCUUGCCGC

-UACAGCGGGGGAGACAUAU--AUCACAGCCUGUCUCGC-----G------CCCGAC-CCUGCUGG-
 ||.|.||..||     |||  .|||.|.||||.||.||     |      |||| | ||.||.|| 
AUAAACCGACGG-----UAUUGUUCAGAUCCUGGCUUGCCAACAGGAGUUCCCCG-CGCCAGCGGGG
  Score=19.1

19.1
No similar sequence found!

AAUGCGCCUGCAGCUCGCGCUCCCGCGCCGAUCCCGAGAGCGUCCGGGCCGCCGUGCGCGAGCGAGGGAGGGCGCGCGCGCGGGGGGGGCGCGCUCGUGAGUGCGGGCCGCGCUCUCGGCGGCGCGCAUGUGCGUGUGUGCUGGCUGCCGGGCUGCCCCGAGCCGGCGGGGAGCCGGUCCGCUCCAGGUGGCGGGCGGCUGGAGCGAGGUGAGGCUGCGGG
221
AAUGCGCCUGCAGCUCGCGCUCCCG-CGCCGAUCCCGAG-----AGCGUCCGG--GCCGCCGUGCGCGAGCGAGGGAG---GGCGCG--CGCGCGGGGGGGGCGCGCUCGUGAGUGCGG---GCCGCGCUCUCGGCGGCGCGCAUGUGCGUGUGUGCUGGCUGCCGGGCUGCCCCGAGCCGGCGGGGA-GCCGGUCCGCUCCAGGUGGCGG-GCGG--CUGGAGCGAGGUGA---GGCUGCGGG-
  .|||   ||||   |.||||||| ||    ||..|||     ||     ||  ||||  ||||   |       ||   |||.||  |.|||          |.|| ||||   |||   |.||| |.|||||      | ||||||       |  .|||              ||||||.|||| |.|||        ||| ||||| |.||  |.||||..|||..|   |.||||.|. 
--GGCG---GCAG---GUGCUCCCGACG---

GGCGCUUCAGG---------U-GG----CCCUGGG-CCGAAAGUACUGACGACA------GACCAGAAGCGGGAAGCUGGGUG-UU---GA-------UGUGU---GAUGC----GACCGGUCUGUCGCAACGUCGUGCCUGCAGGCUCACAGGUU
||.|||  |||         | ||    |||||.. |||.||  ||.|.|||.|      |.|| |||||..| ||  |||.| ||   ||       |.|||   ||.||    || .|||   ||.||.|.|||.||.|||   |||       
GGGGCU--AGGCCGGGGGGUUCGGCGUCCCCUGUAACCGGAA--ACCGCCGAUAUGCCGGGGCC-GAAGCCCG-AG--GGGCGGUUCCCGAAGCCGCCUCUGUAAGGAGGCGGUGGA-GGGU---UCCCACCCUCGGGCGUGC---CUC-------
  Score=43.3

43.29999999999997
No similar sequence found!

GAAUCUUUUCACAAAAGAUUUUAUCCGAUAAUUCUUCCUCGGACAAUUUCGGAUUUGAAUAC
62
G-AAUCUUUUCACAA--AAGAUUUUAUCCGAUAAUUCUUCCUCGGACAAUUUCGGAU--UU-----------GAAUAC
| ||| ||.|.|..|  .|||.|       ||||||..||.||||||  ||.|..||  ||           ||.|.|
GGAAU-UUAUGAGUACGUAGAGU-------AUAAUUAGUCUUCGGAC--UUCCUUAUACUUAUAUACUUAUAGAUUCC
  Score=22.1

22.099999999999987
No similar sequence found!

GGAAGAUAGGCAUGUAGCUUAGCACCUACAUGUCUAUCGCC
41
GGAAGAUAG-------GCAUGUAGCUUAGCA---CCUACAUGUCUAUCG-CC
|||.|||

TypeError: format_alignment() argument after * must be an iterable, not NoneType